# Exploratory Data Analysis (EDA)

This notebook explores the pig iron production data and thermal satellite indices.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src import data_loader as dl

%matplotlib inline
plt.style.use('seaborn-v0_8')

## 1. Production data

In [ ]:
# Set data path
DATA_PATH = "G:/My Drive/Documents/Applications/DBX Commodities 2025/Stage 2 Assignment/Data/Gijon"

# Load production data
production = dl.load_production_data(DATA_PATH)
print(f"Production data shape: {production.shape}")
print(f"Date range: {production.index.min()} to {production.index.max()}")
print(f"\nSummary statistics:")
production.describe()

In [ ]:
# Plot production over time
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 8))

# Time series
ax1.plot(production.index, production['Value'], 'k-', linewidth=2)
ax1.set_ylabel('Production (1000 tons)')
ax1.set_title('Monthly Pig Iron Production')
ax1.grid(True, alpha=0.3)

# Monthly boxplot
production['Month'] = production.index.month
production.boxplot(column='Value', by='Month', ax=ax2)
ax2.set_xlabel('Month')
ax2.set_ylabel('Production (1000 tons)')
ax2.set_title('Seasonal Pattern')
plt.suptitle('')  # Remove default title

# Rolling statistics
production['Value'].plot(ax=ax3, label='Original', alpha=0.7)
production['Value'].rolling(3).mean().plot(ax=ax3, label='3-month MA', linewidth=2)
production['Value'].rolling(6).mean().plot(ax=ax3, label='6-month MA', linewidth=2)
ax3.set_title('Production with Moving Averages')
ax3.set_ylabel('Production (1000 tons)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Thermal index exploration

In [ ]:
# Sample thermal data
sample_date = "20220119"
thermal_data = dl.load_thermal_index(DATA_PATH, "TAI", sample_date)
cloud_mask = dl.load_cloud_mask(DATA_PATH, sample_date)
perimeter_mask = dl.load_perimeter_mask(DATA_PATH)

if thermal_data is not None:
    # Visualise
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Thermal data
    masked_thermal = np.ma.masked_where(~perimeter_mask, thermal_data)
    im1 = axes[0].imshow(masked_thermal, cmap='hot', vmin=0, vmax=2)
    axes[0].set_title(f'TAI - {sample_date}')
    plt.colorbar(im1, ax=axes[0])

    # Cloud mask
    if cloud_mask is not None:
        masked_cloud = np.ma.masked_where(~perimeter_mask, cloud_mask)
        im2 = axes[1].imshow(masked_cloud, cmap='Blues', vmin=0, vmax=3)
        axes[1].set_title('Cloud Mask')
        plt.colorbar(im2, ax=axes[1])

    # Histogram
    valid_pixels = thermal_data[perimeter_mask]
    axes[2].hist(valid_pixels, bins=50, alpha=0.7)
    axes[2].axvline(np.percentile(valid_pixels, 95), color='r',
                   linestyle='--', label='p95')
    axes[2].set_xlabel('TAI Value')
    axes[2].set_ylabel('Pixel Count')
    axes[2].set_title('TAI Distribution')
    axes[2].legend()

    plt.tight_layout()
    plt.show()

## 3. Feature-Production correlation

In [ ]:
# Load pre-computed features if available
try:
    modeling_data = pd.read_csv('../results/csv/modeling_data.csv', index_col=0, parse_dates=True)

    # Calculate correlations
    feature_cols = [col for col in modeling_data.columns if col != 'Value']
    correlations = modeling_data[feature_cols].corrwith(modeling_data['Value']).sort_values(ascending=False)

    # Plot top correlations
    fig, ax = plt.subplots(figsize=(10, 6))
    top_features = correlations.head(15)
    top_features.plot(kind='barh', ax=ax)
    ax.set_xlabel('Correlation with Production')
    ax.set_title('Top 15 Features by Correlation')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Scatter plots of top features
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for i, (feature, corr) in enumerate(top_features.head(4).items()):
        ax = axes[i]
        ax.scatter(modeling_data[feature], modeling_data['Value'], alpha=0.6)
        ax.set_xlabel(feature)
        ax.set_ylabel('Production')
        ax.set_title(f'Correlation: {corr:.3f}')

        # Add trend line
        z = np.polyfit(modeling_data[feature], modeling_data['Value'], 1)
        p = np.poly1d(z)
        ax.plot(sorted(modeling_data[feature]), p(sorted(modeling_data[feature])),
                'r--', alpha=0.8)

    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("No modeling data found. Run the main pipeline first.")

## 4. Cloud coverage analysis

In [ ]:
# Analyse cloud coverage impact
dates = dl.get_available_dates(DATA_PATH)
cloud_stats = []

# Sample 50 random dates
# sample_dates = np.random.choice(dates, min(50, len(dates)), replace=False)
sample_dates = dates

for date in sample_dates:
    cloud_mask = dl.load_cloud_mask(DATA_PATH, date)
    if cloud_mask is not None:
        clear_ratio = np.mean(cloud_mask[perimeter_mask] == 0)
        cloud_stats.append({
            'date': pd.to_datetime(date, format='%Y%m%d'),
            'clear_ratio': clear_ratio
        })

if cloud_stats:
    cloud_df = pd.DataFrame(cloud_stats).set_index('date')

    # Plot cloud coverage over time
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.scatter(cloud_df.index, cloud_df['clear_ratio'], alpha=0.6)
    ax.axhline(y=0.8, color='g', linestyle='--', label='80% clear threshold')
    ax.set_ylabel('Clear Sky Ratio')
    ax.set_xlabel('Date')
    ax.set_title('Cloud Coverage Over Time (Sample)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    print(f"Average clear sky ratio: {cloud_df['clear_ratio'].mean():.2%}")
    print(f"Days with >80% clear: {(cloud_df['clear_ratio'] > 0.8).sum()}")

## Summary

Key findings from EDA:
1. Production shows seasonal variation with lower values in Q4
2. Thermal indices have strong spatial patterns concentrated at industrial components
3. Cloud coverage varies significantly and affects data quality
4. High percentile thermal values (p95, p99) show strong correlation with production